# Intro
Inference notebook for [Hotel-ID starter - similarity - training](https://www.kaggle.com/code/michaln/hotel-id-starter-similarity-training)

Using model and embeddings from the training notebook to generate embeddings for test data and find similar images.

# Setup

In [ ]:
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')

# Imports

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import random
import os
import math

In [ ]:
import huggingface_hub
print(huggingface_hub.__version__)

In [ ]:
#!pip uninstall huggingface_hub -y
#!pip install huggingface_hub==0.4.0
#import huggingface_hub
#print(huggingface_hub.__version__)

In [ ]:
from PIL import Image as pil_image
from tqdm import tqdm

In [ ]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import timm
from sklearn.metrics.pairwise import cosine_similarity

# Global

In [ ]:
SEED = 42
IMG_SIZE = 256
N_MATCHES = 5

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
TRAIN_DATA_FOLDER = "../input/hotelid-2022-train-images-256x256/images/"
TEST_DATA_FOLDER = PROJECT_FOLDER + "test_images/"

In [ ]:
print(os.listdir(PROJECT_FOLDER))

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = pad_image(img)
    return cv2.resize(img, (IMG_SIZE, IMG_SIZE))

In [ ]:
class HotelImageDataset:
    def __init__(self, data, transform=None, data_folder="train_images/"):
        self.data = data
        self.data_folder = data_folder
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + record["image_id"]
        
        image = np.array(open_and_preprocess_image(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        
        return {
            "image" : image,
        }

# Model

In [ ]:
class EmbeddingModel(nn.Module):
    def __init__(self, n_classes=100, embedding_size=64, backbone_name="efficientnet_b0"):
        super(EmbeddingModel, self).__init__()
        
        self.backbone = timm.create_model(backbone_name, num_classes=n_classes, pretrained=False)
        in_features = self.backbone.get_classifier().in_features
        
        self.backbone.classifier = nn.Identity()
        self.embedding = nn.Linear(in_features, embedding_size)
        self.classifier = nn.Linear(embedding_size, n_classes)

    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.embedding(x)
        return x

# Model helper functions

In [ ]:
def predict_tta(args, base_embeddings_df, loader, model, n_matches=5, tta_transforms=3):
    model.eval()
    test_embeds = generate_embeddings(args, test_loader, model, "Generate test embeddings")

    preds = []
    
    with torch.no_grad():
         for query_embeds in tqdm(test_embeds, desc="Similarity - match finding"):
            #input_images = sample['image'].to(args.device)

            tta_outputs = []
            for _ in range(tta_transforms):
                tmp = find_matches(query_embeds, 
                           base_embeddings_df["embeddings"].values, 
                           base_embeddings_df["hotel_id"].values)
                tta_outputs.extend([tmp])

            working = pd.DataFrame(tta_outputs)
            mode_outputs = working.mode()
            print(mode_outputs)
            mode_value = mode_outputs.values.tolist()
            #print(mode_value)
            preds.extend(mode_value)

    #preds = np.argsort(-np.array(preds), axis=1)[:, :n_matches]
    return preds

'''
def predict(args, base_embeddings_df, test_loader, model):
    test_embeds = generate_embeddings(args, test_loader, model, "Generate test embeddings")
    
    preds = []
    for query_embeds in tqdm(test_embeds, desc="Similarity - match finding"):
        tmp = find_matches(query_embeds, 
                           base_embeddings_df["embeddings"].values, 
                           base_embeddings_df["hotel_id"].values)
        preds.extend([tmp])
        
    return preds

def predict(loader, model, n_matches=5):
    preds = []
    with torch.no_grad():
        t = tqdm(loader)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            outputs = model(input)
            outputs = torch.sigmoid(outputs).detach().cpu().numpy()
            preds.extend(outputs)
    
    # get 5 top predictions
    preds = np.argsort(-np.array(preds), axis=1)[:, :5]
    return preds
'''

In [ ]:
def generate_embeddings(args, loader, model, bar_desc="Generating embeds"):
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            output = model(input)
            outputs_all.extend(output.detach().cpu().numpy())
            
    return outputs_all

In [ ]:
def find_matches(query, base_embeds, base_targets, k=N_MATCHES):
    distance_df = pd.DataFrame(index=np.arange(len(base_targets)), data={"hotel_id": base_targets})
    # calculate cosine distance of query embeds to all base embeds
    distance_df["distance"] = cosine_similarity([query], list(base_embeds))[0]
    # sort by distance and hotel_id
    distance_df = distance_df.sort_values(by=["distance", "hotel_id"], ascending=False).reset_index(drop=True)
    # return first 5 different hotel_id_codes
    return distance_df["hotel_id"].unique()[:N_MATCHES]


def predict(args, base_embeddings_df, test_loader, model):
    test_embeds = generate_embeddings(args, test_loader, model, "Generate test embeddings")
    
    preds = []
    for query_embeds in tqdm(test_embeds, desc="Similarity - match finding"):
        tmp = find_matches(query_embeds, 
                           base_embeddings_df["embeddings"].values, 
                           base_embeddings_df["hotel_id"].values)
        preds.extend([tmp])
        
    return preds

# Prepare data

In [ ]:
test_df = pd.DataFrame(data={"image_id": os.listdir(TEST_DATA_FOLDER), "hotel_id": ""}).sort_values(by="image_id")

# Prepare model

In [ ]:
def get_model(backbone_name, checkpoint_path, args):
    model = EmbeddingModel(args.n_classes, args.embedding_size, backbone_name)
        
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint["model"])
    model = model.to(args.device)
    
    return model

In [ ]:
class args:
    batch_size = 64
    num_workers = 2
    embedding_size = 128
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    
    
seed_everything(seed=SEED)

test_dataset = HotelImageDataset(test_df, base_transform, data_folder=TEST_DATA_FOLDER)
test_loader  = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)

In [ ]:
base_embeddings_df = pd.read_pickle('../input/hotel-id-starter-similarity-training/embedding-model-efficientnet_b0-256x256_image-embeddings.pkl')
display(base_embeddings_df.head())

In [ ]:
args.n_classes = base_embeddings_df["hotel_id"].nunique()

model = get_model("efficientnet_b0",
                  "../input/hotel-id-starter-similarity-training/checkpoint-embedding-model-efficientnet_b0-256x256.pt", 
                  args)

# Submission

In [ ]:
%%time

pred = predict_tta(args, base_embeddings_df, test_loader, model, n_matches=5, tta_transforms=3)
#preds = predict(args, base_embeddings_df, test_loader, model)
preds = np.array(pred)
print(type(preds))
preds = preds.astype(int)
print(preds)
# transform array of hotel_ids into string
test_df["hotel_id"] = [str(list(l)).strip("[]").replace(",", "") for l in preds]
#test_df["hotel_id"] = [str(list(preds)).strip("[]").replace(",", "") ]

test_df.to_csv("submission.csv", index=False)
test_df.head()
